In [1]:
########## import some useful package ##########

import time
t1 = time.time()

import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import ripser
import persim

In [2]:
##### setting file path #####

event_path = "/data/mucollider/two_boosted/hhvv_500K/delphes_output.root"
data_file = uproot.open(event_path)

##### read data with features #####

features = ["VLCjetR10N2", "VLCjetR10N2.Eta", "VLCjetR10N2.Phi", "VLCjetR10N2.Particles", "Particle", "Particle.PID", "Particle.Status", "Particle.Eta", "Particle.Phi", "Particle.PT"]
events = []

for feature in features:
    events.append(data_file["Delphes;1"][feature].array())

##### reshape the data #####

events = np.expand_dims(events, axis=-1)
events = events.transpose((1,0,2))
events = np.squeeze(events,axis=(2,))

num_events = len(events)

print("Time:{:^8.4f}(s)".format(time.time()-t1))

/usr/local/lib/python3.8/dist-packages/awkward/array/base.py:394: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return cls.numpy.array(value, copy=False)


Time:59.9278 (s)


In [3]:
event_num = 0
jet_constituents = data_file["Delphes;1"]["VLCjetR10N2.Constituents"].array()[event_num]
jet_particles = data_file["Delphes;1"]["VLCjetR10N2.Particles"].array()[event_num]

print(jet_constituents)
print(jet_particles)

[[612, 512, 614, 531, 616, 621, 519, 528, 615, 515, 523, 619, 521, 609, 499, 503, 505, 525, 618, 507, 509, 493, 610, 497, 696], [643, 564, 638, 671, 636, 633, 623, 625, 547, 549, 552, 566, 629, 631, 558, 560, 562, 568, 545, 627, 554, 540, 542, 536, 538, 697]]
[[109, 161, 208, 220, 239, 158, 239, 240, 235, 238, 208, 219, 234, 159, 163, 224, 237, 125, 154, 157, 153, 156, 203, 224, 236, 188, 171, 155, 111, 187, 109, 291, 321, 288, 239, 240, 338, 341, 208, 327, 289, 224, 340, 125, 284, 287, 283, 286, 320, 312, 299, 285, 111, 311], [86, 193, 168, 88, 87, 138, 180, 92, 229, 230, 197, 167, 213, 214, 198, 178, 200, 199, 192, 139, 177, 142, 141, 166, 144, 248, 314, 297, 88, 87, 273, 180, 92, 332, 333, 315, 296, 213, 214, 316, 305, 200, 199, 313, 139, 304, 276, 275, 295, 277]]


In [12]:
eflowtrack_ID = data_file["Delphes;1"]["EFlowTrack.fUniqueID"].array()[event_num]
eflowphoton_ID = data_file["Delphes;1"]["EFlowPhoton.fUniqueID"].array()[event_num]
eflowNhadron_ID = data_file["Delphes;1"]["EFlowNeutralHadron.fUniqueID"].array()[event_num]
eflow_ID = data_file["Delphes;1"]["EFlow.fUniqueID"].array()[event_num]

total_eflow_ID  = np.concatenate([eflowtrack_ID, eflowphoton_ID, eflowNhadron_ID])
print(eflow_ID==total_eflow_ID)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


In [5]:
np.isin(jet_constituents[1], total_eflow_ID)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False])

In [6]:
eflowtrack_particle = data_file["Delphes;1"]["EFlowTrack.PT"].array()[event_num]
for i in range(len(eflowtrack_particle)):
    print(eflowtrack_particle[i])

15.176668
3.5673594
14.308633
4.3167815
1.5128366
14.731005
10.862693
2.0821433
15.984678
10.800725
9.403575
2.1897748
4.9202037
3.4120812
4.866689
34.94039
19.668285
2.0268662
1.5101599
1.1842505
1.9656774
3.103473
17.680765
4.171065
1.8641999
5.3528824
4.9621024
10.449758
1.2488238
0.73423934
3.150953
0.8571199
0.584118
0.5065386


In [7]:
###### Print Particle Status #####

event_num = 0

print("Index", "\t", "Status", "\t", "PID", "\t", "Eta", "\t\t", "Phi", "\t", "PT")
for particle in range(len(events[event_num][features.index("Particle.Status")])):
    print(particle+1, "\t", events[event_num][features.index("Particle.Status")][particle], "\t\t", events[event_num][features.index("Particle.PID")][particle], "\t", round(events[event_num][features.index("Particle.Eta")][particle],2), "\t\t", round(events[event_num][features.index("Particle.Phi")][particle],2), "\t", round(events[event_num][features.index("Particle.PT")][particle],4))

Index 	 Status 	 PID 	 Eta 		 Phi 	 PT
1 	 4 		 -13 	 999.9 		 0.0 	 0.0
2 	 21 		 -13 	 999.9 		 0.0 	 0.0
3 	 4 		 13 	 -999.9 		 0.0 	 0.0
4 	 21 		 13 	 -999.9 		 0.0 	 0.0
5 	 22 		 25 	 -1.68 		 -2.04 	 301.8967
6 	 22 		 25 	 1.08 		 1.43 	 223.0891
7 	 1 		 14 	 -2.46 		 1.02 	 108.2864
8 	 1 		 -14 	 3.52 		 -0.73 	 66.3812
9 	 23 		 5 	 -2.07 		 -2.19 	 122.3752
10 	 23 		 -5 	 -1.29 		 -1.94 	 181.7301
11 	 51 		 -5 	 -1.26 		 -1.99 	 127.2801
12 	 51 		 21 	 -1.4 		 -1.86 	 56.691
13 	 52 		 5 	 -2.07 		 -2.19 	 120.3749
14 	 51 		 21 	 -1.38 		 -1.85 	 42.1175
15 	 51 		 21 	 -1.44 		 -1.89 	 14.6538
16 	 52 		 5 	 -2.07 		 -2.19 	 120.2956
17 	 51 		 21 	 -1.38 		 -1.87 	 43.077
18 	 51 		 21 	 -1.17 		 -1.78 	 2.2583
19 	 52 		 -5 	 -1.26 		 -1.99 	 124.0397
20 	 23 		 5 	 1.34 		 0.89 	 95.0343
21 	 23 		 -5 	 0.74 		 1.76 	 149.9265
22 	 51 		 5 	 1.29 		 0.88 	 86.1463
23 	 51 		 21 	 1.67 		 1.02 	 9.8655
24 	 52 		 -5 	 0.74 		 1.76 	 148.5337
25 	 51 		 -5 	 0.75 	

In [8]:
data_file["Delphes;1"].show()

Event                      TStreamerInfo              asdtype('>i4')
Event.fUniqueID            TStreamerBasicType         asjagged(asdtype('>u4'))
Event.fBits                TStreamerBasicType         asjagged(asdtype('>u4'))
Event.Number               TStreamerBasicType         asjagged(asdtype('>i8'))
Event.ReadTime             TStreamerBasicType         asjagged(asdtype('>f4'))
Event.ProcTime             TStreamerBasicType         asjagged(asdtype('>f4'))
Event.ProcessID            TStreamerBasicType         asjagged(asdtype('>i4'))
Event.MPI                  TStreamerBasicType         asjagged(asdtype('>i4'))
Event.Weight               TStreamerBasicType         asjagged(asdtype('>f4'))
Event.CrossSection         TStreamerBasicType         asjagged(asdtype('>f4'))
Event.CrossSectionError    TStreamerBasicType         asjagged(asdtype('>f4'))
Event.Scale                TStreamerBasicType         asjagged(asdtype('>f4'))
Event.AlphaQED             TStreamerBasicType         asjagged